In [38]:
import numpy as np
import pandas as pd
import torch
import time
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

In [28]:
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

seed = 1337
set_seed(seed)

In [29]:
import os
import sys

notebook_dir = os.getcwd()

root_dir = os.path.abspath(os.path.join(notebook_dir, "../../"))
sys.path.append(root_dir)

In [30]:
from dotenv import load_dotenv

load_dotenv()

from service import EnvironmentSecretService
from coral import DataSourceFactory
from core.models.datasource_type import DataSourceType
from core.models.protocol_type import ProtocolType

In [31]:
DEFAULT_DATASOUCE = DataSourceType.BYBIT
DEFAULT_SYMBOLS = ["XRPUSDT"]
SLEEP_INTERVAL = 1
DOM = 10

datasource = DataSourceFactory(EnvironmentSecretService())
datasource.register_rest_exchange(DEFAULT_DATASOUCE)

In [39]:
def plot_order_book(bids, asks):
    bids_df = pd.DataFrame(bids, columns=['price', 'quantity'])
    asks_df = pd.DataFrame(asks, columns=['price', 'quantity'])
    
    bids_df = bids_df.sort_values(by='price', ascending=False)
    
    asks_df = asks_df.sort_values(by='price', ascending=True)

    bids_df['cum_quantity'] = bids_df['quantity'].cumsum()
    asks_df['cum_quantity'] = asks_df['quantity'].cumsum()

    clear_output(wait=True)

    sns.set_style("whitegrid")
    
    plt.figure(figsize=(10, 6))
    
    sns.lineplot(x=bids_df['price'], y=bids_df['cum_quantity'], label='Bids', color='green', linewidth=2)
    sns.lineplot(x=asks_df['price'], y=asks_df['cum_quantity'], label='Asks', color='red', linewidth=2)
    
    plt.xlabel('Price', fontsize=12)
    plt.ylabel('Cumulative Quantity', fontsize=12)
    plt.title('Order Book Depth Chart', fontsize=14)
    
    plt.legend()
    
    plt.show()

In [40]:
ds = datasource.create(DEFAULT_DATASOUCE, ProtocolType.REST)

symbols = ds.fetch_future_symbols()
symbol = [symbol for symbol in symbols if symbol.name in DEFAULT_SYMBOLS]

while True:
    bids, asks, timestamp = ds.fetch_order_book(symbol[0], DOM)
    
    plot_order_book(bids, asks)
    
    time.sleep(SLEEP_INTERVAL)

KeyboardInterrupt: 